In [ ]:
%%time

import pandas as pd

import glob

files = glob.glob('../data/2022/yellow_taxi/*')

df = pd.read_parquet(files,columns=["trip_distance","store_and_fwd_flag","total_amount"])

grouped_df = df.groupby('store_and_fwd_flag')['total_amount'].sum()

print(grouped_df)

In [3]:
%%time

import polars as pl

df_lazy = pl.scan_parquet("../data/*/yellow_taxi/*").group_by("store_and_fwd_flag").agg(
    pl.col("total_amount").sum())

result = df_lazy.collect(streaming=True)

print(result)

shape: (11, 2)
┌────────────────────┬──────────────┐
│ store_and_fwd_flag ┆ total_amount │
│ ---                ┆ ---          │
│ str                ┆ f64          │
╞════════════════════╪══════════════╡
│ 1                  ┆ 7.7806e6     │
│ 0.0                ┆ 7.7422e7     │
│ 1.0                ┆ 1.6408e6     │
│ Y                  ┆ 2.0194e8     │
│ null               ┆ 5.2716e9     │
│ …                  ┆ …            │
│ 0                  ┆ 4.4005e8     │
│ nan                ┆ 5.7917e7     │
│ N                  ┆ 1.6116e10    │
│ 2                  ┆ 60.08        │
│                    ┆ 5851.9       │
└────────────────────┴──────────────┘
CPU times: user 33.6 s, sys: 3.7 s, total: 37.3 s
Wall time: 17.4 s


In [1]:
%%time

import pandas as pd

import glob

files = glob.glob('../data/*/yellow_taxi/*')

df = pd.read_parquet(files,columns=["trip_distance","store_and_fwd_flag",
                                    "total_amount"])

grouped_result = df.groupby('store_and_fwd_flag').agg(
    {'total_amount':'sum'}
).query('total_amount > 1000')

print(grouped_result)

                    total_amount
store_and_fwd_flag              
                    5.851900e+03
0                   4.400451e+08
0.0                 7.742174e+07
1                   7.780618e+06
1.0                 1.640831e+06
N                   1.611615e+10
Y                   2.019364e+08
nan                 5.791688e+07
CPU times: user 1min 23s, sys: 35.6 s, total: 1min 59s
Wall time: 6min 24s


In [2]:
%%time

import polars as pl

df_lazy = pl.scan_parquet("../data/*/yellow_taxi/*").group_by("store_and_fwd_flag").agg(
    pl.col("total_amount").sum().alias("total_amount_sum")
).filter(pl.col("total_amount_sum") >= 100000)

result = df_lazy.collect(streaming=True)

print(result)

shape: (8, 2)
┌────────────────────┬──────────────────┐
│ store_and_fwd_flag ┆ total_amount_sum │
│ ---                ┆ ---              │
│ str                ┆ f64              │
╞════════════════════╪══════════════════╡
│ 0                  ┆ 4.4005e8         │
│ N                  ┆ 1.6116e10        │
│ Y                  ┆ 2.0194e8         │
│ 0.0                ┆ 7.7422e7         │
│ nan                ┆ 5.7917e7         │
│ null               ┆ 5.2716e9         │
│ 1.0                ┆ 1.6408e6         │
│ 1                  ┆ 7.7806e6         │
└────────────────────┴──────────────────┘
CPU times: user 33.8 s, sys: 3.76 s, total: 37.6 s
Wall time: 17.3 s
